#### Importing libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import pickle

#### getting my playlists

In [2]:
top_100_songs = pd.read_csv('top_100_songs.csv')
playlist_8_cluster = pd.read_csv('playlist_8_cluster.csv')

#### connect to spotify

In [63]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [64]:
secrets_file = open("../secrets.txt","r")

In [65]:
string = secrets_file.read()

In [66]:
string.split('\n')

['clientid:e9c8174323214a1d84247cc1e2018d9d',
 'clientsecret:11813fb826984d6ab23b42023d4a5b28']

In [67]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [68]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

### create a sample recommender

In [69]:
track = input("Enter a song that you love: ")

Enter a song that you love: simarik


In [70]:
if top_100_songs['title'].str.lower().str.contains(str(track).lower()).any() == True:
        print('here what I found for you! ')
        print(top_100_songs['title'].sample())
else:
        print('continue below !!!')

continue below !!!


### improve the simple recommender

### searching the track

In [71]:
results = sp.search(q="track:" + track, type="track")

In [72]:
song_uri = results['tracks']['items'][0]['uri'].split(':')[2]
song_uri

'6vEL7E8TGzrxuxbrgdlkLL'

In [73]:
# get the audio features for that song
song_features = sp.audio_features(song_uri)
song_features

[{'danceability': 0.775,
  'energy': 0.818,
  'key': 7,
  'loudness': -8.687,
  'mode': 1,
  'speechiness': 0.164,
  'acousticness': 0.0518,
  'instrumentalness': 0,
  'liveness': 0.114,
  'valence': 0.826,
  'tempo': 97.06,
  'type': 'audio_features',
  'id': '6vEL7E8TGzrxuxbrgdlkLL',
  'uri': 'spotify:track:6vEL7E8TGzrxuxbrgdlkLL',
  'track_href': 'https://api.spotify.com/v1/tracks/6vEL7E8TGzrxuxbrgdlkLL',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6vEL7E8TGzrxuxbrgdlkLL',
  'duration_ms': 235218,
  'time_signature': 3}]

In [74]:
new = pd.DataFrame.from_dict(song_features)

In [75]:
y_pred = new.drop(columns=['type', 'id' , 'uri', 'track_href', 'analysis_url'])
y_pred

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.775,0.818,7,-8.687,1,0.164,0.0518,0,0.114,0.826,97.06,235218,3


In [76]:
### SCALING???

In [77]:
# from sklearn.preprocessing import StandardScaler
# target = StandardScaler().fit_transform(y_pred)

# # Now, all features will have the same weight.
# pd.DataFrame(target,columns=y_pred.columns).head()

#### predict the song cluster

In [78]:
kmeans = pickle.load(open('kmeans_8_cluster.p', 'rb'))

In [79]:
look_for = kmeans.predict(y_pred)

In [80]:
look_for

array([2])

In [81]:
display(playlist_8_cluster.loc[playlist_8_cluster.clusters == look_for[0]].sample(1))

,title,artist,url,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,id,uri,track_href,analysis_url,duration_ms,time_signature,clusters
3835,Crank That (Soulja Boy) [Travis Barker Remix],Soulja Boy,https://open.spotify.com/track/5WeiB5JDBy6T5QQ...,0.671,0.531,8.0,-5.551,1.0,0.0467,0.0307,...,0.365,0.28,140.013,5WeiB5JDBy6T5QQxtj09zb,spotify:track:5WeiB5JDBy6T5QQxtj09zb,https://api.spotify.com/v1/tracks/5WeiB5JDBy6T...,https://api.spotify.com/v1/audio-analysis/5Wei...,247013.0,4.0,2
